In [1]:
%%capture
!pip install dowhy==0.6

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import dowhy
import dowhy.api
import dowhy.datasets
import dowhy.plotter

# Causal Inference with DoWhy

By Alessio Zanga and Fabio Stella

## Abstract

This notebook represents a quick-start guide to causal inference using the [DoWhy](https://github.com/microsoft/dowhy) package.

## Generate a Dataset

DoWhy allows the generation of a dataset by specifying a set of parameters.

In [4]:
dataset = dowhy.datasets.linear_dataset(
    beta = 1,
    num_samples = int(1e4),
    num_common_causes = 1,
    num_frontdoor_variables = 1,
    treatment_is_binary = True,
    outcome_is_binary = True
)
list(dataset.keys())

['df',
 'treatment_name',
 'outcome_name',
 'common_causes_names',
 'instrument_names',
 'effect_modifier_names',
 'frontdoor_variables_names',
 'dot_graph',
 'gml_graph',
 'ate']

It will return a dictionary with a Pandas DataFrame, along with a set of variables names and the true causal graph used for the sampling process.

In [5]:
data, [treatment_name], outcome_name, _, _, _, _, _, causal_graph, ace = dataset.values()

After unpacking the values of the dictionary, it is possible to observe the assigned names to the data columns:

In [6]:
treatment_name, outcome_name

('v0', 'y')

It also return the true average causal effect:

In [7]:
ace

In [8]:
data

,FD0,W0,v0,y
0,-0.101090,-1.671592,False,True
1,-1.272862,-1.377525,True,False
2,-0.761740,-0.427812,False,False
3,0.469911,-2.095372,False,True
4,0.002100,-1.576471,False,False
...,...,...,...,...
9995,-0.406907,-2.617531,False,False
9996,-0.140561,0.028068,False,False
9997,0.227541,-2.028969,False,True
9998,-0.347695,-0.819795,True,False


## Construct a Model

The second step consists in constructing a causal model using the true graph.

In [9]:
model = dowhy.CausalModel(
    data = data,
    treatment = treatment_name,
    outcome = outcome_name,
    graph = causal_graph
)

It is not a proper SCM as we seen in previous sessions. It is more like a container for the data and the graph.

In [10]:
model.view_model(layout="dot")
# from IPython.display import Image, display    # Run these lines if you are on local.
# display(Image(filename="causal_model.png"))

This will enable the identification step by using a couple of handful methods.

## Identify the Estimand

The estimand identification tests for both backdoor and frontdoor criterion.

In [11]:
estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(estimand)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                     
─────(Expectation(y|W0))
d[v₀]                   
Estimand assumption 1, Unconfoundedness: If U→{v0} and U→y then P(y|v0,W0,U) = P(y|v0,W0)

### Estimand : 2
Estimand name: iv
No such variable found!

### Estimand : 3
Estimand name: frontdoor
Estimand expression:
Expectation(Derivative(y, [FD0])*Derivative([FD0], [v0]))
Estimand assumption 1, Full-mediation: FD0 intercepts (blocks) all directed paths from v0 to y.
Estimand assumption 2, First-stage-unconfoundedness: If U→{v0} and U→{FD0} then P(FD0|v0,U) = P(FD0|v0)
Estimand assumption 3, Second-stage-unconfoundedness: If U→{FD0} and U→y then P(y|FD0, v0, U) = P(y|FD0, v0)



## Estimate the Average Causal Effect

Once the estimand has been identified, we can proceed to the last estimation step.

In [12]:
methods = [
    "backdoor.linear_regression",
    "backdoor.propensity_score_stratification",
    "backdoor.propensity_score_weighting",
    "backdoor.propensity_score_matching",
]

We will iterate over the provided methods and compare the results.

In [13]:
estimates = {
    method: model.estimate_effect(estimand, method_name=method)
    for method in methods
}

In [14]:
for estimate in estimates.values():
    print(estimate)
    break

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                     
─────(Expectation(y|W0))
d[v₀]                   
Estimand assumption 1, Unconfoundedness: If U→{v0} and U→y then P(y|v0,W0,U) = P(y|v0,W0)

## Realized estimand
b: y~v0+W0
Target units: ate

## Estimate
Mean value: 0.019287447461825702



In [15]:
for method, estimate in estimates.items():
    print(f"""
    Estimation Method: {method},
    Estimated ACE: {estimate.value:.4},
    Relative Error: {(abs((estimate.value-ace)/ace*100)):.4}%
    """.lstrip("\n"))

    Estimation Method: backdoor.linear_regression,
    Estimated ACE: 0.01929,
    Relative Error: 33.95%
    
    Estimation Method: backdoor.propensity_score_stratification,
    Estimated ACE: 0.01722,
    Relative Error: 41.04%
    
    Estimation Method: backdoor.propensity_score_weighting,
    Estimated ACE: 0.01807,
    Relative Error: 38.12%
    
    Estimation Method: backdoor.propensity_score_matching,
    Estimated ACE: 0.0153,
    Relative Error: 47.6%
    


## Evaluate the Results

With the estimates obtained we can setup a round of hypothesis testing.

In [16]:
refuters = [
    "bootstrap_refuter",
    "data_subset_refuter",
    "dummy_outcome_refuter",
    "placebo_treatment_refuter",
    "random_common_cause",
]

We will take the backdoor with linear regression as estimate of reference.

In [17]:
refuteds = {
    refuter: model.refute_estimate(estimand, estimates["backdoor.linear_regression"], method_name=refuter)
    for refuter in refuters
}

Some "refuters", as they are called in DoWhy, may return a list of tests.

In [18]:
for refuter in refuteds.values():
    print(refuter if type(refuter) != list else refuter[0])

Refute: Bootstrap Sample Dataset
Estimated effect:0.019287447461825702
New effect:0.01895048816849586
p value:0.45999999999999996

Refute: Use a subset of data
Estimated effect:0.019287447461825702
New effect:0.018970895924973037
p value:0.45999999999999996

Refute: Use a Dummy Outcome
Estimated effect:0
New effect:-0.0027021875754324664
p value:0.45999999999999996

Refute: Use a Placebo Treatment
Estimated effect:0.019287447461825702
New effect:-7.852935162945162e-05
p value:0.5

Refute: Add a Random Common Cause
Estimated effect:0.019287447461825702
New effect:0.019271989912366783



Your results may vary. Try to experiment with different graph configurations by altering the numbers of variables in the generation step.

Have fun!